### Overview:
This code implements one of the multiple ways of multi-model RAG. It extracts and processes text and images from PDFs, utilizing a multi-modal Retrieval-Augmented Generation (RAG) system for summarizing and retrieving content for question answering.

### Key Components:
   - **PyMuPDF**: For extracting text and images from PDFs.
   - **Gemini 1.5-flash model**: To summarize images and tables.
   - **Cohere Embeddings**: For embedding document splits.
   - **Chroma Vectorstore**: To store and retrieve document embeddings.
   - **LangChain**: To orchestrate the retrieval and generation pipeline.

### Diagram:
   <img src="https://github.com/NirDiamant/RAG_Techniques/blob/main/images/multi_model_rag_with_captioning.svg?raw=1" alt="Reliable-RAG" width="300">

### Motivation:
Efficiently summarize complex documents to facilitate easy retrieval and concise responses for multi-modal data.

### Method Details:
   - Text and images are extracted from the PDF using PyMuPDF.
   - Summarization is performed on extracted images and tables using Gemini.
   - Embeddings are generated via Cohere for storage in Chroma.
   - A similarity-based retriever fetches relevant sections based on the query.

### Benefits:
   - Simplified retrieval from complex, multi-modal documents.
   - Streamlined Q&A process for both text and images.
   - Flexible architecture for expanding to more document types.

### Implementation:
   - Documents are split into chunks with overlap using a text splitter.
   - Summarized text and image content are stored as vectors.
   - Queries are handled by retrieving relevant document segments and generating concise answers.

### Summary:
The project enables multi-modal document processing and retrieval, providing concise, relevant responses by combining state-of-the-art LLMs and vector-based retrieval systems.

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [ ]:
# Install required packages
!pip install langchain langchain-community pillow pymupdf python-dotenv

In [ ]:
!pip install langchain_cohere

In [5]:
import fitz  # PyMuPDF
from PIL import Image
import io
import os
#from dotenv import load_dotenv

import google.generativeai as genai
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_cohere import ChatCohere, CohereEmbeddings

#load_dotenv()

False

In [16]:
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['COHERE_API_KEY'] = userdata.get('COHERE_API_KEY')

### Download the "Attention is all you need" paper

In [6]:
!wget https://arxiv.org/pdf/1706.03762
!mv 1706.03762 attention_is_all_you_need.pdf

--2025-06-17 11:12:45--  https://arxiv.org/pdf/1706.03762
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.3.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘1706.03762’

1706.03762          100%[===================>]   2.11M  6.27MB/s    in 0.3s    

2025-06-17 11:12:45 (6.27 MB/s) - ‘1706.03762’ saved [2215244/2215244]



### Data Extraction

In [7]:
text_data = []
img_data = []

In [8]:
with fitz.open('attention_is_all_you_need.pdf') as pdf_file:
    # Create a directory to store the images
    if not os.path.exists("extracted_images"):
        os.makedirs("extracted_images")

    # Loop through every page in the PDF
    for page_number in range(len(pdf_file)):
        page = pdf_file[page_number]

        # Get the text on page
        text = page.get_text().strip()
        text_data.append({"response": text, "name": page_number+1})
        # Get the list of images on the page
        images = page.get_images(full=True)

        # Loop through all images found on the page
        for image_index, img in enumerate(images, start=0):
            xref = img[0]  # Get the XREF of the image
            base_image = pdf_file.extract_image(xref)  # Extract the image
            image_bytes = base_image["image"]  # Get the image bytes
            image_ext = base_image["ext"]  # Get the image extension

            # Load the image using PIL and save it
            image = Image.open(io.BytesIO(image_bytes))
            image.save(f"extracted_images/image_{page_number+1}_{image_index+1}.{image_ext}")

In [13]:
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

### Image Captioning

In [14]:
for img in os.listdir("extracted_images"):
    image = Image.open(f"extracted_images/{img}")
    response = model.generate_content([image, "You are an assistant tasked with summarizing tables, images and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text or image:"])
    img_data.append({"response": response.text, "name": img})

### Vectostore

In [17]:
# Set embeddings
embedding_model = CohereEmbeddings(model="embed-english-v3.0")

# Load the document
docs_list = [Document(page_content=text['response'], metadata={"name": text['name']}) for text in text_data]
img_list = [Document(page_content=img['response'], metadata={"name": img['name']}) for img in img_data]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=400, chunk_overlap=50
)

doc_splits = text_splitter.split_documents(docs_list)
img_splits = text_splitter.split_documents(img_list)

In [ ]:
!pip install chromadb

In [20]:
# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits + img_splits, # adding the both text and image splits
    collection_name="multi_model_rag",
    embedding=embedding_model,
)

retriever = vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={'k': 1}, # number of documents to retrieve
            )

### Query

In [25]:
queries=["What is the main contribution of this paper?",
        "How does the Transformer model differ from recurrent and convolutional neural networks?",
        "What datasets were used for training and evaluating the Transformer model?",
        "What are the advantages of using self-attention?",
        "Can you explain Figure 1?"]

In [31]:
for query in queries:
    print(query + "\n")
    docs = retriever.invoke(query)
    print("Relevant documents:" + docs[0].page_content + "\n\n")

What is the main contribution of this paper?

Relevant documents:[25] Mitchell P Marcus, Mary Ann Marcinkiewicz, and Beatrice Santorini. Building a large annotated
corpus of english: The penn treebank. Computational linguistics, 19(2):313–330, 1993.
[26] David McClosky, Eugene Charniak, and Mark Johnson. Effective self-training for parsing. In
Proceedings of the Human Language Technology Conference of the NAACL, Main Conference,
pages 152–159. ACL, June 2006.
[27] Ankur Parikh, Oscar Täckström, Dipanjan Das, and Jakob Uszkoreit. A decomposable attention
model. In Empirical Methods in Natural Language Processing, 2016.
[28] Romain Paulus, Caiming Xiong, and Richard Socher. A deep reinforced model for abstractive
summarization. arXiv preprint arXiv:1705.04304, 2017.
[29] Slav Petrov, Leon Barrett, Romain Thibaux, and Dan Klein. Learning accurate, compact,
and interpretable tree annotation. In Proceedings of the 21st International Conference on
Computational Linguistics and 44th Annual Me

In [22]:
docs = retriever.invoke(query)

### Output

In [33]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge.
Use three-to-five sentences maximum and keep the answer concise."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>"),
    ]
)

# LLM
llm = ChatCohere(model="command-r-plus", temperature=0)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
for query in queries:
  print("Query:\n"+ query + "\n")
  generation = rag_chain.invoke({"documents":docs[0].page_content, "question": query})
  print("Answer: \n"+ generation + "\n\n")

Query:
What is the main contribution of this paper?

Answer: 
The main contribution of this paper is the introduction of a new method for encoding positional information in neural sequence transduction models, using a sinusoidal function to generate positional encodings.


Query:
How does the Transformer model differ from recurrent and convolutional neural networks?

Answer: 
The Transformer model differs from recurrent and convolutional neural networks in its ability to process sequential data in parallel, allowing for more efficient training on long sequences. It also uses self-attention mechanisms to capture long-range dependencies, which are challenging for recurrent and convolutional networks.


Query:
What datasets were used for training and evaluating the Transformer model?

Answer: 
The question cannot be answered based on the provided text.


Query:
What are the advantages of using self-attention?

Answer: 
Self-attention allows for learning long-range dependencies, which is a